<h1>OpenRice Data Analyse</h1>
<h2> Part 1 - Restaurant Info Scraping</h2>
Steps
Search restaurants in OpenRice website with different filters (cuisine type, location, etc.) and collect the data into MySQL database

Related Website: https://www.openrice.com/zh/hongkong

In [1]:
import pandas as pd
import json
from bs4 import BeautifulSoup
import mysql.connector
import requests
import re
import os
import threading
import time

<h2>Step 1: Collect Meta data for later scripping</h2>

In [29]:
#Gather openrice metadata for web scripping
url = 'https://www.openrice.com/api/v2/metadata/region/all?uiLang=zh&uiCity=hongkong'
my_headers = {'user-agent': 'my-app/0.0.1'}
response = requests.get(url, headers = my_headers)
info=json.loads(response.content.decode())

In [30]:
column_names_districts = ['districtId','eng_name','zh_name','searchKey']
df_districts = pd.DataFrame(columns = column_names_districts)
for i in info['districts']:
    new_row = {'districtId': i['districtId'], 'eng_name': i['nameLangDict']['en']
               ,'zh_name':i['nameLangDict']['tc-tw'],'searchKey': i['searchKey']}
    df_districts = df_districts.append(new_row, ignore_index=True)

In [31]:
df_districts.head()

,districtId,eng_name,zh_name,searchKey
0,1999,Hong Kong Island,香港島,districtId=1999
1,2999,Kowloon,九龍,districtId=2999
2,3999,New Territories,新界,districtId=3999
3,4999,Outlying Islands,離島,districtId=4999
4,1008,Western District,西環,districtId=1008


In [32]:
df_districts.to_csv("./data/openrice_districts.csv", mode='w',encoding="utf-8-sig")

In [33]:
column_names_categories = ['categoryTypeId','categoryType','categoryId','category_eng_name','category_zh_name','searchKey']
df_categories = pd.DataFrame(columns = column_names_categories)
for k,v in info['categories'].items():
    categoryType = k
    for a in v:
        new_row = {'categoryTypeId': a['categoryTypeId']
                   ,'categoryType': categoryType
                   ,'categoryId': a['categoryId']
                   ,'category_eng_name': a['callNameLangDict']['en']
                   ,'category_zh_name':a['callNameLangDict']['tc-tw']
                   , 'searchKey' : a['searchKey']}
        df_categories = df_categories.append(new_row, ignore_index=True)

In [34]:
df_categories.head()

,categoryTypeId,categoryType,categoryId,category_eng_name,category_zh_name,searchKey
0,1,cuisine,4000,western,西式,cuisineId=4000
1,1,cuisine,2009,japanese,日本菜,cuisineId=2009
2,1,cuisine,6000,international,多國菜,cuisineId=6000
3,1,cuisine,3010,french,法國菜,cuisineId=3010
4,1,cuisine,4001,american,美國菜,cuisineId=4001


In [104]:
df_categories.to_csv("./data/openrice_categories.csv", mode='w',encoding="utf-8-sig")

<h2>Step 2: Creating scrape function</h2>

In [7]:
def scrape(url):

    try: 
        response = requests.get(url, headers = my_headers)
        assert response.status_code == 200
    except: 
        print("Cannot access " + url)
    soup = BeautifulSoup(response.content.decode(), 'html.parser')

    zh_name = "" if soup.find('span', {'class': "name"},{'itemprop': "name"}) is None else soup.find('span', {'class': "name"},{'itemprop': "name"}).text
    eng_name = "" if soup.find('div', {"class": "smaller-font-name"}) is None else soup.find('div', {"class": "smaller-font-name"}).text
    star = "" if soup.find('div', {'class': 'header-score'}) is None else soup.find('div', {'class': 'header-score'}).text
    bookmark = "" if soup.find('div', {'class': 'header-bookmark-count js-header-bookmark-count'}) \
                    is None else soup.find('div', {'class': 'header-bookmark-count js-header-bookmark-count'}).text
    district = "" if soup.find('div', {'class': 'header-poi-district dot-separator'}) is None else soup.find('div', {'class': 'header-poi-district dot-separator'}).text.strip()
    price = "" if soup.find('div', {'class': 'header-poi-price dot-separator'}) is None else soup.find('div', {'class': 'header-poi-price dot-separator'}).text.strip()
    
    categories_list = ['western','japanese','international','french','american','chiu-chow','sichuan','hunan','guangdong','shanghai',
                       'beijing','peruvian','hong-kong-style','middle-eastern','brazilian','cuba','taiwan','indonesian','indian',
                       'jingchuanhu','northeastern','eastern-europe','african','hakka','malaysian','singaporean','british','egyptian',
                       'thai','dutch','jewish','vietnamese','swiss','portuguese','country-food','mongolia','german','moroccan',
                       'australian','korean','turkish','belgian','greek','nepalese','mediterranean','spanish','argentinian','philippines',
                       'austrian','italian','irish','lebanon','mexican','sri-lankan','xinjiang','burmese','guangxi','zhejiang','hubei',
                       'yunnan','fujian','guizhou','shandong','shanxi-jin','shanxi-shan','huaiyang','shunde']
    cuisine = []
    dish = []
    for a in soup.find('div', {'class': 'header-poi-categories dot-separator'}).text.strip().split('\n'):
        cuisine.append(a.lower()) if a.lower() in categories_list else dish.append(a.lower())
    cuisine = '/'.join(cuisine)
    dish = '/'.join(dish)

    address = soup.find('section', {'class': 'address-section'}).find_all('div', {'class': 'content'})
    eng_address = address[0].text.strip()
    zh_address = address[1].text.strip() if len(address) > 1 else ""

    emoji_smile = int(soup.find_all('div', {"class": "score-div"})[0].text)
    emoji_ok = int(soup.find_all('div', {"class": "score-div"})[1].text)
    emoji_cry = int(soup.find_all('div', {"class": "score-div"})[2].text)

    special = soup.find('span', {'class': 'poi-with-other-status'})
    special = "Operating" if special is None else special.text

    #review
    if re.search(r'\d+',soup.find('div', {'class': 'main-menu table-center'}).find_all('li')[1].text):
        review_cnt = int(re.search(r'\d+',soup.find('div', {'class': 'main-menu table-center'}).find_all('li')[1].text).group())
    else:
        review_cnt = 0

    #photo
    if re.search(r'\d+',soup.find('div', {'class': 'main-menu table-center'}).find_all('li')[2].text):
        photo_cnt = int(re.search(r'\d+',soup.find('div', {'class': 'main-menu table-center'}).find_all('li')[2].text).group())
    else:
        photo_cnt = 0

    latest_review = "" if len(soup.find_all('span', {'itemprop': 'datepublished'})) == 0 \
                            else soup.find_all('span', {'itemprop': 'datepublished'})[0].text

    lat_long = soup.find('div', {'class': 'mapview-container'})
    if lat_long is not None:
        lat = lat_long.get("data-latitude")
        long = lat_long.get("data-longitude")
    else:
        lat = ""
        long = ""

    new_row = {'zh_name':zh_name,'eng_name':eng_name,'star':star,'bookmark':bookmark,
                'district':district,'price':price,'cuisine':cuisine,'dish':dish,
                'eng_address':eng_address,'zh_address':zh_address,'emoji_smile':emoji_smile,
                'emoji_ok':emoji_ok,'emoji_cry':emoji_cry,'special':special,'review_cnt':review_cnt,
                'photo_cnt':photo_cnt,'latest_review':latest_review,'lat':lat,'long':long}
    return new_row


<h2>Step 3: Go though all URL to get related infomation</h2>

In [ ]:
def scrape_restaurants(category, district, urls):
    column_names_restaurant=['zh_name','eng_name','star','bookmark','district','price','cuisine','dish','eng_address','zh_address','emoji_smile','emoji_ok','emoji_cry','special','review_cnt','photo_cnt','latest_review','lat','long']
    df_restaurant = pd.DataFrame(columns = column_names_restaurant)
    for url in urls:
        result = scrape(url)
        if result is not None:
            df_restaurant = df_restaurant.append(result, ignore_index=True)
        
    df_restaurant.to_parquet('./data/by_district/by_category_by_district/by_category_openrice_restaurants_{0}_{1}.parquet'.format(category, district))
    print("------------------------------saved to ./data/by_district/by_category_by_district/by_category_openrice_restaurants_{0}_{1}.parquet------------------------------------------------".format(category, district))
    return

def process_urls(category, district, urls):
    batch_size = 100
    num_batches = len(urls) // batch_size + (1 if len(urls) % batch_size > 0 else 0)
    for i in range(num_batches):
        start = i * batch_size
        end = min((i + 1) * batch_size, len(urls))
        batch_urls = urls[start:end]
        scrape_restaurants(category, district, batch_urls)

# Create a list of threads
threads = []
for category in df_categories[df_categories['categoryType'] == 'cuisine']['searchKey']:
    for district in df_districts[(df_districts['districtId'] != 1999 )& (df_districts['districtId'] != 2999) & (df_districts['districtId'] != 3999) & (df_districts['districtId'] != 4999)]['searchKey']:
        search_url = 'https://www.openrice.com/en/hongkong/restaurants?{0}&{1}&page={2}'
        my_headers = {'user-agent': 'my-app/0.0.1'}
        try:
            response = requests.get(search_url.format(category, district, 1), headers=my_headers)
            time.sleep(2)
            assert response.status_code == 200
            soup = BeautifulSoup(response.content.decode(), 'html.parser')
            print('processing {0} & {1}  '.format(category, district))
        except:
            print('error in reading {0} & {1}  '.format(category, district))
            continue
        page = soup.findAll('div', {'class': "content-cell-wrapper"})
        if len(page) == 0:
            print('no restaurants found for {0} & {1}  '.format(category, district))
            continue
        res_set = set()
        for p in page:
            if p.find("span", {"class": "pois-restaurant-list-cell-sponsored"}) is None:
                res_url = p.find("a", href=True)
                full_url = "https://www.openrice.com" + res_url['href']
                res_set.add(full_url)
        urls = list(res_set)
        t = threading.Thread(target=process_urls, args=(category, district, urls))
        threads.append(t)
        t.start()

# Wait for all threads to finish
for t in threads:
    t.join()

print("All threads have finished")

<h2> Step 4: Write to MySQL database </h2>

In [95]:
df_temp = pd.read_parquet("./data/by_category_by_district/{0}".format(os.listdir("./data/by_category_by_district")[0]))
df = pd.DataFrame(columns=df_temp.columns)
df['districtId'] = ""
df.head()

,zh_name,eng_name,star,bookmark,district,price,cuisine,dish,eng_address,zh_address,emoji_smile,emoji_ok,emoji_cry,special,review_cnt,photo_cnt,latest_review,lat,long,districtId


In [149]:
#add districtId & read all files into one df
districtId = re.findall(r'districtId=(-?\d+)',",".join(os.listdir("./data/by_category_by_district")))
districtId = set(districtId)
for district in districtId:
    df_district = pd.DataFrame(columns=df_temp.columns)
    for files in os.listdir("./data/by_category_by_district"):
        if files.find('districtId={0}'.format(district)) != -1:
            df_files = pd.read_parquet("./data/by_category_by_district/{0}".format(files))
            df_district = pd.concat([df_district,df_files])
    df_district['districtId'] = district
    df = pd.concat([df,df_district])
    #print("distrist {0} done".format(district))
df.to_parquet('./data/openrice_restaurants.parquet')

In [155]:
#updating cuisine & dish type
mask1 = (df['cuisine'] == '') & ((df['dish'].str.startswith('chiu chow/'))|(df['dish'].str.startswith('chiu chow')))
mask2 = (df['cuisine'] != '') & ((df['dish'].str.startswith('chiu chow/'))|(df['dish'].str.startswith('chiu chow')))

df.loc[mask1, 'dish'] = df.loc[mask1, 'dish'].str.replace('chiu chow/?', '', regex=True)
df.loc[mask1, 'cuisine'] = 'chiu-chow'
df.loc[mask2, 'dish'] = df.loc[mask2, 'dish'].str.replace('chiu chow/?', '', regex=True)
df.loc[mask2, 'cuisine'] = df.loc[mask2, 'cuisine'] + '/chiu-chow'

df['dish'] = df['dish'].str.replace(' ', '-')
df['cuisine'] = df['cuisine'].str.replace(' ', '-')

In [156]:
d_mapping = dict(zip(df_categories['category_eng_name'], df_categories['categoryType'].str[0] \
                     + '-' + df_categories['categoryId'].astype(str)))

def map_names_to_numbers(category_eng_name):
    category_list = category_eng_name.split('/')
    mapped_list = []
    for category in category_list:
        if category in d_mapping:
            mapped_list.append(str(d_mapping[category]))
    mapped_string = '/'.join(mapped_list)
    return mapped_string


def seperate_name(category_code,category):
    category_list = category_code.split('/')
    extracted_list = [s[2:] for s in category_list if s.startswith(category)]
    mapped_string = '/'.join(extracted_list)
    return mapped_string

df['category'] = df[['dish', 'cuisine']].apply(lambda x: '/'.join(map(str, x)), axis=1).map(map_names_to_numbers)
df['cuisine_clean'] = df['category'].apply(lambda x: seperate_name(x,'c-'))
df['dish_clean'] = df['category'].apply(lambda x: seperate_name(x,'d-'))
df['amenity_clean'] = df['category'].apply(lambda x: seperate_name(x,'a-'))

In [157]:
df = df.rename(columns={'long': 'lng'})
df.head()

,zh_name,eng_name,star,bookmark,district,price,cuisine,dish,eng_address,zh_address,...,review_cnt,photo_cnt,latest_review,lat,lng,districtId,category,cuisine_clean,dish_clean,amenity_clean
0,鶴咀阿蘇涼茶,,3.5,50,Shek O,Below $50,,hong-kong-style/herbal-tea/sweet-soup,"Hok Tsui Tsuen, Shek O",石澳鶴咀村,...,7,29,2021-11-20,22.2087424714314,114.24596786499,1007,c-1004/d-1013/d-1202,1004,1013/1202,
1,Daphne,,4.5,2368,Shek O,$101-200,italian/western,pizza,"G/F, 330 Shek O Village, Shek O",石澳石澳村330號地舖,...,30,298,2023-03-09,22.2304218229094,114.252182543278,1007,d-1022/c-3006/c-4000,3006/4000,1022,
2,石澳基記,,,48,Shek O,Below $50,international,,"Shek O Beach, Shek O",石澳石澳正灘,...,2,13,2016-07-17,22.2305556,114.2519444,1007,c-6000,6000,,
3,昌盛海鮮飯店,Cheong Sing Restaurant,3.5,91,Shek O,$101-200,guangdong,seafood/noodles/rice-noodles/dim-sum/stir-fry/...,"G/F, 566 Shek O Village,, Shek O",石澳石澳村566號地下,...,3,14,2011-05-01,22.23018,114.253165,1007,d-1009/d-1036/a-1001/a-1034/c-1002,1002,1009/1036,1001/1034
4,Shining Stone,,4,612,Shek O,$101-200,french,,石澳石澳村452號,,...,21,84,2021-07-04,22.2301815,114.2520753,1007,c-3010,3010,,


In [158]:
# Set up database connection parameters
config = {
    'user': 'root',
    'password': '94927419',
    'host': 'localhost',
    'database': 'openrice'
}

# Create a connection object
cnx = mysql.connector.connect(**config)

# Create a cursor object
cursor = cnx.cursor(buffered=True)

In [159]:
table_name = 'raw_restaurants'

columns = ', '.join(df.columns)
types = ', '.join([f"{col} varchar(255)" for col in df.columns])
query = f"CREATE TABLE {table_name} ({types})"

cursor.execute(query)

for _, row in df.iterrows():
    values = tuple(row)
    query = f"INSERT INTO {table_name} ({columns}) VALUES {values}"
    cursor.execute(query)

cnx.commit()

In [64]:
table_name = 'raw_districts'

columns = ', '.join(df_districts.columns)
types = ', '.join([f"{col} varchar(255)" for col in df_districts.columns])
query = f"CREATE TABLE {table_name} ({types})"

cursor.execute(query)

for _, row in df_districts.iterrows():
    values = tuple(row)
    query = f"INSERT INTO {table_name} ({columns}) VALUES {values}"
    cursor.execute(query)

cnx.commit()

In [65]:
table_name = 'raw_categories'

columns = ', '.join(df_categories.columns)
types = ', '.join([f"{col} varchar(255)" for col in df_categories.columns])
query = f"CREATE TABLE {table_name} ({types})"

cursor.execute(query)

for _, row in df_categories.iterrows():
    values = tuple(row)
    query = f"INSERT INTO {table_name} ({columns}) VALUES {values}"
    cursor.execute(query)

cnx.commit()

In [160]:
cursor.execute('select * from openrice.raw_restaurants')
result = cursor.fetchall()
result[0]

('鶴咀阿蘇涼茶',
 '',
 '3.5',
 '50',
 'Shek O',
 'Below $50',
 '',
 'hong-kong-style/herbal-tea/sweet-soup',
 'Hok Tsui Tsuen, Shek O',
 '石澳鶴咀村',
 '5',
 '0',
 '1',
 'Operating',
 '7',
 '29',
 '2021-11-20',
 '22.2087424714314',
 '114.24596786499',
 '1007',
 'c-1004/d-1013/d-1202',
 '1004',
 '1013/1202',
 '')

In [161]:
# Close the cursor and connection
cursor.close()
cnx.close()